In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import geopandas
import rioxarray
from shapely.geometry import mapping

import os
import sys

# To avoid warning messages
import warnings
warnings.filterwarnings('ignore')

In [71]:
parent_data_dir = r'C:/Users/s1834371/Documents/PLUMES/validation/mooring_test_4/'  #file directory of in-situ data

# Count number of files to list through
file_list = []
for (dir_path, dir_names, file_names) in os.walk(os.path.normpath(parent_data_dir)):
        file_list.extend(file_names)
    
len_files = len(file_list) #length of list of all files
count = 1 #for loop code
file_dict = {}
file_dict['directory'] = []
file_dict['tile'] = []

for path, subdirs, files in os.walk(os.path.normpath(parent_data_dir)):
    for name in files:
        if name.endswith(".nc"):
            print('\n====================')
            print('filename:', name)

            insitu_data = xr.open_dataset(os.path.join(path, name))  # open in-situ data
            insitu_data = insitu_data.sel(TIME=slice('2020-01-01', '2020-12-31')) # select time period of interest
            print(insitu_data)

            ##### looping through each timestep and retrieving lat, lon, time, depth and temperature data #####
            for index in range(len(insitu_data.TIME)):
                print('timestep:', index+1, 'of', len(insitu_data.TIME))
                insitu_data_t = insitu_data.isel(TIME=index) #select time
                insitu_data_t = insitu_data_t.isel(LONGITUDE=index) #select longitude
                insitu_data_t = insitu_data_t.isel(LATITUDE=index) #select latitude
                insitu_data_t = insitu_data_t.isel(POSITION=index) #select position
                insitu_data_t.load() # solves the issue but makes it a lot slower

                if 'DC_REFERENCE' in insitu_data_t.variables:
                    insitu_data_t = insitu_data_t.drop_vars(['DC_REFERENCE']) # drop unnecessary variables - required to be able to subset to depth
                if 'DC_REFERENCE' in insitu_data_t.variables:
                    insitu_data_t = insitu_data_t.drop_vars(['DIRECTION'])  # drop unnecessary variables

                # for var in insitu_data_t.variables:
                #     print(insitu_data_t[var].dtype, var)
                insitu_data_upper_3m_t = insitu_data_t.where((insitu_data_t['DEPH'] >= 0.0) & (insitu_data_t['DEPH'] < 3.0), drop=True)
                # print(insitu_data_upper_3m_t.DEPH_QC.values)
                # print(insitu_data_upper_3m_t.TIME_QC.values)
                # sys.exit()
      
  

                try: # try to subsetting to upper 3m, if no data exsists move on to next timestep
                    insitu_data_upper_3m_t = insitu_data_upper_3m_t.where((insitu_data_upper_3m_t['TEMP_QC'] == 1) & (insitu_data_upper_3m_t['DEPH_QC'] == 1) | (insitu_data_upper_3m_t['DEPH_QC'] == 7) & (insitu_data_upper_3m_t['TIME_QC'] <= 2) & (insitu_data_upper_3m_t['POSITION_QC'] <= 2), drop=True)  # good data only     
                    #insitu_data_upper_3m_t = insitu_data_upper_3m_t.where((insitu_data_upper_3m_t['DEPH'] == insitu_data_upper_3m_t['DEPH'].min()), drop=True) #select only the uppermost depth
                    insitu_data_upper_3m_t['TEMP'] = insitu_data_upper_3m_t['TEMP'].mean() # take the mean of the temperature values

                    insitu_df = insitu_data_upper_3m_t.to_dataframe() # convert to pandas dataframe for saving
                    insitu_df['ID'] = insitu_data_upper_3m_t.attrs['id'] # add ID variable

                    if index == 0: # if not the first timestep append to dataframe
                        insitu_df_all = insitu_df
                    else:
                        insitu_df_all = insitu_df_all.append(insitu_df)
                
                except:
                    pass
            
            try:
                insitu_df_all.to_csv(os.path.join(path, name[:-3] + '_surface.csv')) # save as csv
            except:
                pass



filename: GL_TS_MO_4400050.nc
<xarray.Dataset>
Dimensions:       (TIME: 3593, LATITUDE: 173823, LONGITUDE: 173823,
                   POSITION: 173823, DEPTH: 2)
Coordinates:
  * TIME          (TIME) datetime64[ns] 2020-10-14T13:09:59.999999744 ... 202...
  * LATITUDE      (LATITUDE) float32 46.7 46.7 46.7 46.7 ... 46.7 46.7 46.7 46.7
  * LONGITUDE     (LONGITUDE) float32 -56.18 -56.18 -56.18 ... -56.18 -56.18
Dimensions without coordinates: POSITION, DEPTH
Data variables: (12/31)
    TIME_QC       (TIME) float32 ...
    POSITION_QC   (POSITION) float32 ...
    DC_REFERENCE  (TIME) object ...
    DEPH          (TIME, DEPTH) float32 ...
    DEPH_QC       (TIME, DEPTH) float32 ...
    TEMP          (TIME, DEPTH) float64 ...
    ...            ...
    VTM02         (TIME, DEPTH) float64 ...
    VTM02_QC      (TIME, DEPTH) float32 ...
    VTPK          (TIME, DEPTH) float64 ...
    VTPK_QC       (TIME, DEPTH) float32 ...
    VZMX          (TIME, DEPTH) float64 ...
    VZMX_QC       (TIME,

SystemExit: 

In [ ]:
parent_data_dir = r'C:/Users/s1834371/Documents/PLUMES/validation/mooring_test_4/'  #file directory of in-situ data

# Count number of files to list through
file_list = []
for (dir_path, dir_names, file_names) in os.walk(os.path.normpath(parent_data_dir)):
        file_list.extend(file_names)
    
len_files = len(file_list) #length of list of all files
count = 1 #for loop code
file_dict = {}
file_dict['directory'] = []
file_dict['tile'] = []

for path, subdirs, files in os.walk(os.path.normpath(parent_data_dir)):
    for name in files:
        if name.endswith(".nc"):
            print('\n====================')
            print('filename:', name)

            insitu_data = xr.open_dataset(os.path.join(path, name))  # open in-situ data
            insitu_data = insitu_data.sel(TIME=slice('2020-01-01', '2020-12-31')) # select time period of interest
            print(insitu_data)

            ##### looping through each timestep and retrieving lat, lon, time, depth and temperature data #####
            for index in range(len(insitu_data.TIME)):
                print('timestep:', index+1, 'of', len(insitu_data.TIME))
                insitu_data_t = insitu_data.isel(TIME=index) #select time
                insitu_data_t = insitu_data_t.isel(LONGITUDE=index) #select longitude
                insitu_data_t = insitu_data_t.isel(LATITUDE=index) #select latitude
                insitu_data_t = insitu_data_t.isel(POSITION=index) #select position
                insitu_data_t.load() # solves the issue but makes it a lot slower

                if 'DC_REFERENCE' in insitu_data_t.variables:
                    insitu_data_t = insitu_data_t.drop_vars(['DC_REFERENCE']) # drop unnecessary variables - required to be able to subset to depth
                if 'DC_REFERENCE' in insitu_data_t.variables:
                    insitu_data_t = insitu_data_t.drop_vars(['DIRECTION'])  # drop unnecessary variables

                # for var in insitu_data_t.variables:
                #     print(insitu_data_t[var].dtype, var)
                insitu_data_upper_3m_t = insitu_data_t.where((insitu_data_t['DEPH'] >= 0.0) & (insitu_data_t['DEPH'] < 3.0), drop=True)
                # print(insitu_data_upper_3m_t.DEPH_QC.values)
                # print(insitu_data_upper_3m_t.TIME_QC.values)
                # sys.exit()
      
  

                try: # try to subsetting to upper 3m, if no data exsists move on to next timestep
                    insitu_data_upper_3m_t = insitu_data_upper_3m_t.where((insitu_data_upper_3m_t['TEMP_QC'] == 1) & (insitu_data_upper_3m_t['DEPH_QC'] == 1) | (insitu_data_upper_3m_t['DEPH_QC'] == 7) & (insitu_data_upper_3m_t['TIME_QC'] <= 2) & (insitu_data_upper_3m_t['POSITION_QC'] <= 2), drop=True)  # good data only     
                    #insitu_data_upper_3m_t = insitu_data_upper_3m_t.where((insitu_data_upper_3m_t['DEPH'] == insitu_data_upper_3m_t['DEPH'].min()), drop=True) #select only the uppermost depth
                    insitu_data_upper_3m_t['TEMP'] = insitu_data_upper_3m_t['TEMP'].mean() # take the mean of the temperature values

                    insitu_df = insitu_data_upper_3m_t.to_dataframe() # convert to pandas dataframe for saving
                    insitu_df['ID'] = insitu_data_upper_3m_t.attrs['id'] # add ID variable

                    if index == 0: # if not the first timestep append to dataframe
                        insitu_df_all = insitu_df
                    else:
                        insitu_df_all = insitu_df_all.append(insitu_df)
                
                except:
                    pass
            
            try:
                insitu_df_all.to_csv(os.path.join(path, name[:-3] + '_surface.csv')) # save as csv
            except:
                pass



filename: GL_TS_MO_4400050.nc
<xarray.Dataset>
Dimensions:       (TIME: 3593, LATITUDE: 173823, LONGITUDE: 173823,
                   POSITION: 173823, DEPTH: 2)
Coordinates:
  * TIME          (TIME) datetime64[ns] 2020-10-14T13:09:59.999999744 ... 202...
  * LATITUDE      (LATITUDE) float32 46.7 46.7 46.7 46.7 ... 46.7 46.7 46.7 46.7
  * LONGITUDE     (LONGITUDE) float32 -56.18 -56.18 -56.18 ... -56.18 -56.18
Dimensions without coordinates: POSITION, DEPTH
Data variables: (12/31)
    TIME_QC       (TIME) float32 ...
    POSITION_QC   (POSITION) float32 ...
    DC_REFERENCE  (TIME) object ...
    DEPH          (TIME, DEPTH) float32 ...
    DEPH_QC       (TIME, DEPTH) float32 ...
    TEMP          (TIME, DEPTH) float64 ...
    ...            ...
    VTM02         (TIME, DEPTH) float64 ...
    VTM02_QC      (TIME, DEPTH) float32 ...
    VTPK          (TIME, DEPTH) float64 ...
    VTPK_QC       (TIME, DEPTH) float32 ...
    VZMX          (TIME, DEPTH) float64 ...
    VZMX_QC       (TIME,

SystemExit: 

In [125]:
import geopandas
import rioxarray as rio
from shapely.geometry import mapping


def ROI_select(DATA, ROI_dir, invert=None): # clips raster to given shapefile area, option to invert or not
        print('Clipping DATA to ROI')
        shapefile_dir = ROI_dir
        glacier_shape = geopandas.read_file(shapefile_dir)
        DATA.rio.set_spatial_dims(x_dim="LONGITUDE", y_dim="LATITUDE", inplace=True)
        DATA.rio.write_crs("epsg:4326", inplace=True)
        DATA = DATA.rio.clip(glacier_shape.geometry.apply(mapping), glacier_shape.crs, drop=True, all_touched=True, invert=invert)
        return DATA

In [23]:
import glob
import xarray as xr
import dask.array as da
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar
import os
import sys
import numpy as np

client = Client(n_workers=4, threads_per_worker=2, memory_limit='8GB')

parent_data_dir = r'C:/Users/s1834371/Documents/PLUMES/validation/mooring_test_4/'  #file directory of in-situ data

# Get list of .nc files
file_list = glob.glob(parent_data_dir + '**/*.nc', recursive=True)

for file in file_list:
    print('\n====================')
    print('filename:', os.path.basename(file))

    insitu_data = xr.open_dataset(file)  # open in-situ data
    chunk_size = int(len(insitu_data.TIME) / 10)

    insitu_data = insitu_data.chunk({'TIME':chunk_size,'LONGITUDE': chunk_size, 'LATITUDE':chunk_size, 'POSITION':chunk_size})  # chunks need to be as big as possible

    insitu_data = insitu_data.isel(DEPTH=0) # select time period of interest
    insitu_data = insitu_data[['TEMP', 'TEMP_QC', 'DEPH', 'DEPH_QC', 'TIME_QC', 'POSITION_QC', 'LONGITUDE', 'LATITUDE', 'TIME', 'POSITION']]


    print(insitu_data)

    depth_condition = (insitu_data['DEPH'] >= 0.0) & (insitu_data['DEPH'] < 3.0) 
    temp_qc_condition = (insitu_data['TEMP_QC'] == 1)
    deph_qc_condition = (insitu_data['DEPH_QC'] == 1) | (insitu_data['DEPH_QC'] == 7)
    time_qc_condition = (insitu_data['TIME_QC'] <= 2)
    position_qc_condition = (insitu_data['POSITION_QC'] <= 2)

    # Setting conditions for dataset
    insitu_data = insitu_data.where(depth_condition, drop=True)
    insitu_data = insitu_data.where(temp_qc_condition, drop=True)
    insitu_data = insitu_data.where(deph_qc_condition, drop=True)
    insitu_data = insitu_data.where(time_qc_condition, drop=True)
    insitu_data = insitu_data.where(position_qc_condition, drop=True)

    insitu_data_df = insitu_data.to_dataframe() # convert to pandas dataframe for saving
    insitu_data_df['ID'] = insitu_data.attrs['id'] # add ID variable
    print(insitu_data_df)
    insitu_data_df.to_csv(file[:-3] + '_surface_v3.csv') # save as csv

    print('\n AFTER FILTERING')
    print(insitu_data_df)
    sys.exit()



c:\Users\s1834371\Miniconda3\envs\JIF_main\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61961 instead
  warnings.warn(



filename: GL_TS_MO_4400050.nc
<xarray.Dataset>
Dimensions:      (TIME: 173823, POSITION: 173823, LONGITUDE: 173823,
                  LATITUDE: 173823)
Coordinates:
  * LONGITUDE    (LONGITUDE) float32 -56.18 -56.18 -56.18 ... -56.18 -56.18
  * LATITUDE     (LATITUDE) float32 46.7 46.7 46.7 46.7 ... 46.7 46.7 46.7 46.7
  * TIME         (TIME) datetime64[ns] 2011-11-30T14:39:59.999999744 ... 2023...
  * POSITION     (POSITION) int64 0 1 2 3 4 ... 173819 173820 173821 173822
Data variables:
    TEMP         (TIME) float64 dask.array<chunksize=(17382,), meta=np.ndarray>
    TEMP_QC      (TIME) float32 dask.array<chunksize=(17382,), meta=np.ndarray>
    DEPH         (TIME) float32 dask.array<chunksize=(17382,), meta=np.ndarray>
    DEPH_QC      (TIME) float32 dask.array<chunksize=(17382,), meta=np.ndarray>
    TIME_QC      (TIME) float32 dask.array<chunksize=(17382,), meta=np.ndarray>
    POSITION_QC  (POSITION) float32 dask.array<chunksize=(17382,), meta=np.ndarray>
Attributes: (12/49)
 

c:\Users\s1834371\Miniconda3\envs\JIF_main\lib\site-packages\dask\array\core.py:4806: PerformanceWarning: Increasing number of chunks by factor of 11
  result = blockwise(
c:\Users\s1834371\Miniconda3\envs\JIF_main\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

2023-12-15 19:55:19,604 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:52355', name: 0, status: running, memory: 0, processing: 0>
2023-12-15 19:55:19,730 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:51428', name: 3, status: running, memory: 0, processing: 0>
2023-12-15 19:55:19,736 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:52355'.
2023-12-15 19:55:20,003 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:51428'.
2023-12-15 19:55:22,384 - distributed.nanny - WARNING - Restarting worker
